In [1]:
# mount data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
# import linear regression model
from sklearn.linear_model import LinearRegression
# import regression metrics needed from sklearn
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [4]:
# load the dataset
df = pd.read_csv('/content/sales_predictions (3).csv')
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [12]:
df.duplicated().any()

# check for any duplicate data

False

In [13]:
df.isnull().sum()

# check for null values in data 

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [16]:
df[df['Item_Weight'].isnull()]

# want to investigate these null values a little 

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
18,DRI11,NaN,Low Fat,0.034238,Hard Drinks,113.2834,OUT027,1985,Medium,Tier 3,Supermarket Type3,2303.6680
21,FDW12,NaN,Regular,0.035400,Baking Goods,144.5444,OUT027,1985,Medium,Tier 3,Supermarket Type3,4064.0432
23,FDC37,NaN,Low Fat,0.057557,Baking Goods,107.6938,OUT019,1985,Small,Tier 1,Grocery Store,214.3876
29,FDC14,NaN,Regular,0.072222,Canned,43.6454,OUT019,1985,Small,Tier 1,Grocery Store,125.8362
...,...,...,...,...,...,...,...,...,...,...,...,...
8485,DRK37,NaN,Low Fat,0.043792,Soft Drinks,189.0530,OUT027,1985,Medium,Tier 3,Supermarket Type3,6261.8490
8487,DRG13,NaN,Low Fat,0.037006,Soft Drinks,164.7526,OUT027,1985,Medium,Tier 3,Supermarket Type3,4111.3150
8488,NCN14,NaN,Low Fat,0.091473,Others,184.6608,OUT027,1985,Medium,Tier 3,Supermarket Type3,2756.4120
8490,FDU44,NaN,Regular,0.102296,Fruits and Vegetables,162.3552,OUT019,1985,Small,Tier 1,Grocery Store,487.3656


In [17]:
filtering_item_weight = df['Item_Weight']

item_weight_avg = filtering_item_weight.mean()

filtering_item_weight.fillna(item_weight_avg, inplace= True)

df.isna().sum()

filtering_data_size_1 = df['Outlet_Location_Type'] == 'Tier 1'

filtering_data_size_2 = df['Outlet_Location_Type'] == 'Tier 2'

filtering_data_size_3 = df['Outlet_Location_Type'] == 'Tier 3'

outlet_size = df['Outlet_Size']

df.loc[filtering_data_size_1 & (df['Outlet_Size'].isna()), 'Outlet_Size'] = 'Small'

df.loc[filtering_data_size_2 & (df['Outlet_Size'].isna()), 'Outlet_Size'] = 'Medium'

df.loc[filtering_data_size_3 & (df['Outlet_Size'].isna()), 'Outlet_Size'] = 'Large'

df.isnull().sum()

# this is how we will take care of the null values. Use average weight for tne null values in weight. If outlet location type tier is 1 then for outlet size that is null make it small. 
# For tier 2 outlet location type if it has outlet size null make it medium

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [19]:
dummy_df = pd.get_dummies(df, columns = ['Item_Fat_Content'], drop_first=True)

dummy_df

dummy_df = pd.get_dummies(df, columns = ['Item_Type'], drop_first=True)

dummy_df = pd.get_dummies(df, columns = ['Outlet_Size'], drop_first=True)

dummy_df = pd.get_dummies(df, columns = ['Outlet_Type'], drop_first=True)

dummy_df

# we will convert categorical variables into dummy variables

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Item_Outlet_Sales,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,3735.1380,1,0,0
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,443.4228,0,1,0
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,2097.2700,1,0,0
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Large,Tier 3,732.3800,0,0,0
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,994.7052,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,2778.3834,1,0,0
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,Medium,Tier 2,549.2850,1,0,0
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,1193.1136,1,0,0
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,1845.5976,0,1,0


In [20]:
feature_characteristics = ['Item_Identifier',
                 'Item_Weight',
                 'Item_Fat_Content',
                 'Item_Visibility',
                 'Item_Type',
                 'Item_MRP',
                 'Outlet_Identifier',
                 'Outlet_Establishment_Year',
                 'Outlet_Size',
                 'Outlet_Location_Type',
                ]            
# Get columns for features matrix
df.loc[:, feature_characteristics]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Large,Tier 3
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3
...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,Medium,Tier 2
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3


In [21]:
# Convert to numpy array
X = df.loc[:, feature_characteristics].to_numpy()

In [22]:
# Pandas series to convert to NumPy Array
df.loc[:, 'Item_Outlet_Sales']

0       3735.1380
1        443.4228
2       2097.2700
3        732.3800
4        994.7052
          ...    
8518    2778.3834
8519     549.2850
8520    1193.1136
8521    1845.5976
8522     765.6700
Name: Item_Outlet_Sales, Length: 8523, dtype: float64

In [23]:
y = df.loc[:, 'Item_Outlet_Sales'].values

In [24]:
# Import the TTS from sklearn
from sklearn.model_selection import train_test_split
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [25]:
from sklearn.linear_model import LinearRegression

In [26]:
reg = LinearRegression()

reg.fit(X_train,y_train)

ValueError: ignored

In [27]:
# or to predict all values on the training data
train_house = reg.predict(X_train)
# predict the values in the test set
test_house = reg.predict(X_test)

NotFittedError: ignored

In [ ]:
train_score = reg.score(X_train, y_train)
print(train_score)

In [ ]:
test_score = reg.score(X_test, y_test)
print(test_score)

In [ ]:
from sklearn.metrics import mean_squared_error

print('Training RMSE:', np.sqrt(mean_squared_error(y_train, reg.predict(X_train))))

print('Training RMSE:', np.sqrt(mean_squared_error(y_test, reg.predict(X_test))))